Professor Shekhar

Rita Kurban

In [1]:
# Get all the necessary packages
from queue import PriorityQueue
import random

## PuzzleNode Class

In [2]:
class PuzzleNode(object):
    """PuzzleNode is a class that captures the state of
    the 8-puzzle and its generalizations, as well
    as the elements needed to implement an A∗ search.
    """

    def __init__(self, n=3, init_state = None, parent = None):
        """Initializes an instance of a class with a
        certain number of tiles in a raw and the initial
        state and parent node that can be identified by the user.
        Each object is characterized by the number of tiles in
        a row, its parent, and a 1-D grid.
        """
        self.n = n
        self.parent = parent
        if init_state == None:
            self.grid = self.gen_state()
        self.check_state(init_state)
        self.grid = [item for sublist in init_state for item in sublist]
    
    def neighbor_states(self):
        """ This function returns a list of 2 to 4 neighboring states."""
        state = self.get_state()
        neighbors = []
        for i in range(self.n**2):
            new_puzzle_node = PuzzleNode(self.n, state)
            if new_puzzle_node.move(i / self.n, i % self.n):
                # Add the parent node
                new_puzzle_node.parent = self
                neighbors.append(new_puzzle_node)
        return neighbors
    
    def check_state(self, init_state):
        """ Checks whether the state specified by the user has the right format."""
        assert len(init_state) == self.n, "Wrong number of rows"
        for i in range(len(init_state)):
            assert len(init_state[i]) == self.n, "Wrong number of columns in row {}".format(i + 1)
        st = [item for sublist in init_state for item in sublist]
        goal = range(self.n**2)
        assert len(goal) == len(st) and set(goal).issubset(st), "Wrong tile labels"

    def gen_state(self):
        """ Generates a 1-D grid if not specified by the user."""
        num = self.n**2
        return random.sample(range(0,num), num)
        
    def __str__(self):
        """ Prints out 2-D states."""
        s = ''
        for i in range(self.n):
            for j in range(self.n):
                s += str(self.grid[i * self.n + j]) + " "
            s += "\n"
        return s
    
    def __repr__(self):
        """ Node representation."""
        s = ''
        for i in range(self.n):
            for j in range(self.n):
                s += str(self.grid[i * self.n + j]) + " "
            s += "\n"
        return s

    def ok(self, coord):
        """ Checks whether the coordinates specified by the user are ok"""
        return coord >= 0 and coord < self.n ** 2
            
    def move(self, x, y):
        """ Checks whether it's possible to move the tile and moves it. """
        if x < 0 or x >= self.n or y < 0 or y >= self.n:
            return False
        coord = x * self.n + y
        if self.grid[coord] == 0:
            return False
        directions = [1, self.n, -1, -self.n]
        for i in directions:
            new_coord = coord + i
            if self.ok(new_coord):
                cond = abs(new_coord / self.n - coord / self.n) + abs(new_coord % self.n  - coord % self.n) 
                if self.grid[new_coord] == 0 and cond == 1:
                    self.grid[new_coord] = self.grid[coord]
                    self.grid[coord] = 0
                    return True
                
    def get_state(self):
        """ Outputs a 2-D representation of states"""
        state =[[] for i in range(self.n)]
        for i in range(self.n):
            for j in range(self.n):
                state[i].append(self.grid[i * self.n + j])
        return state
    
    def __eq__(self, other):
        """ 
        Makes it possible to compare nodes directly.
        Nodes are considered equal if they are in the same state."""
        return self.grid == other.grid
    
    def __hash__(self):
        """ Hash function is required for the set operations."""
        return hash(tuple(self.grid))
    
    def solvable(self):
        n_invers = 0
        for i in range(len(self.grid)):
            for j in range(i, len(self.grid)):
                if self.grid[i] > self.grid[j] and self.grid[j] != 0:
                    n_invers += 1
        return n_invers % 2 == 0

 In the last function of the cell above, I'm checking whether the state is solvable. I used this resource: https://www.youtube.com/watch?v=0pTciAl6Wc4 to find out that for the state to be solvable, it should have an even number of inversions. Inversion is when a tile precedes other lower number tile.

## Heuristics

In [3]:
def misplaced(state, n=3):
    """ Calculates the number of misplaced tiles."""
    goal = range(n**2)
    st =  [item for sublist in state for item in sublist]
    count = 0
    for i in range(len(st)):
        if st[i] != goal[i]:
            count += 1
    return count

def manhattan(state, n=3):
    """ 
    Calculates the Manhattan distance. Returns summed
    distance / 2 since 2 tiles are moved each time.
    """
    count = 0
    for i in range(len(state)):
        for j in range(len(state[i])):
            row = state[i][j]/n
            column = state[i][j] % n
            count += abs(i-row) + abs(j-column)
    return count / 2

def inversions(state, n=3):
    """ Calculates the number of inversions.
    I think this heuristic is closer to the actual
    number of moves because it considers where tiles
    are relative to each other and acts similar to humans
    who try to position each tile in its correct position.
    """
    n_invers = 0
    puzzle = PuzzleNode(n, state)
    for i in range(len(puzzle.grid)):
        for j in range(i, len(puzzle.grid)):
            if puzzle.grid[i] > puzzle.grid[j]:
                n_invers += 1
    return n_invers 

    # Heuristics handle
heuristics = [misplaced, manhattan, inversions]

## Optimal Solution

In [4]:
def solvePuzzle(n, state, heuristic,prnt = False):
    """ 
    A function that checks the state of the puzzle and
    implements A* search using different heuristics.
    Inputs:
    n - the puzzle dimension (i.e. n x n board)
    state - the starting (scrambled) state of the puzzle,
    provided as a list of lists, with the blank space
    represented by the number 0. 
    heuristic - a handle to a heuristic function.
    prnt - a boolean value that indicates whether or
    not to print the solution.
    Outputs:
    steps - the number of steps to optimally reach the goal
    state from the initial state
    frontierSize - the frontier size.
    err - an error code (details given in the next step)
    """
    # Check if the state is ok
    try:
        puzzle = PuzzleNode(n, state)
    except:
        return 0, 0, -1
    
    # Check if the state is solvable
    if not puzzle.solvable():
        return 0, 0, -2
    
    frontier = PriorityQueue()
    frontier.put((heuristic(state), 0, puzzle))
    visited = set()
    total_steps = 0
    while not frontier.empty():
        total_steps += 1
        item = frontier.get()
        current_puzzle = item[2]
        min_steps = item[1]
        if heuristic(current_puzzle.get_state()) == heuristic([[0,1,2], [3,4,5], [6,7,8]]):
            if prnt:
                # Reconstruct the optimal path using parent nodes
                optimal_path = [current_puzzle.get_state()]
                while current_puzzle.parent:
                    optimal_path.append((current_puzzle.parent).get_state())
                    current_puzzle = current_puzzle.parent
                print("The number of steps algorithm required to reach the goal is %d.\n"% total_steps)
                print("Optimal Path to Goal:\n")
                for st in optimal_path[::-1]:
                    print(st)
                print ("Frontier size: %d"%frontier.qsize())
            return min_steps, frontier.qsize(), 0
        # Get neighboring states
        list_of_neighbors = current_puzzle.neighbor_states()
        for i, neighbor in enumerate(list_of_neighbors):
            # Check if neighbor was visited
            if neighbor not in visited:
                state = neighbor.get_state()
                # The nodes are sorted in the frontier according to h() + g().
                frontier.put((heuristic(state) + min_steps + 1, (min_steps + 1), neighbor))
                visited.add(neighbor)

In [5]:
init_states = [[[5,7,6],[2,4,3],[8,1,0]],
               [[7,0,8],[4,6,1],[5,3,2]], 
               [[2,3,7],[1,8,0],[6,5,4]]]
for state in init_states:
    for heuristic in heuristics:
        print solvePuzzle(3,state, heuristic, False)


(28, 21608, 0)
(28, 19719, 0)
(28, 4422, 0)
(25, 12856, 0)
(25, 9761, 0)
(29, 4498, 0)
(17, 558, 0)
(17, 470, 0)
(17, 649, 0)


All the heuristics managed to find the same shortest paths to the goal state. Manhattan significantly outperformed the number of misplaced tiles, while the number of inversions outperformed the other heuristics in most of the cases. It required a bigger frontier size in the third size which might mean that this heuristic is not admissible and doesn't behave optimally in all the cases. However, since this heuristic comes closer to the actual number of moves needed it outperforms the other heuristics almost all the time.